In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

In [3]:
import torch
import segmentation_models_pytorch as smp
from omegaconf import OmegaConf
from torch import nn

from dl.inferer import *
from dl.lightning_model import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/l

In [4]:
segmodel = smp.FPN(encoder_name="resnext50_32x4d", classes=2, decoder_merge_policy='cat')

criterion = nn.CrossEntropyLoss(
    ignore_index = -100,
    reduction = 'none'
)

PATH = "/home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/epoch=17.ckpt"

conf = OmegaConf.create(
    {
        'dirs': {
            'test_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/test_Kumar.pytable",
            'train_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/train_Kumar.pytable"
        },
        
        'dataloader_args': {
            'batch_size':6,
            'n_classes':2
        },
        
        'optimizer_args':{
            'lr':0.001,
            'encoder_lr':0.0005,
            'weight_decay':0.0003,
            'encoder_weight_decay':0.00003
        },
        
        'scheduler_args': {
            'factor':0.25,
            'patience':2
        },
    
        'loss_args' : {
            'edge_weight' : 1.1,
        }

    }
)

model = SegModel(model=segmodel, criterion=criterion, hparams=conf)
checkpoint = torch.load(PATH, map_location = lambda storage, loc : storage)

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [5]:
ray.init()

2020-07-09 18:39:33,656	INFO resource_spec.py:212 -- Starting Ray with 15.04 GiB memory available for workers and up to 7.53 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
/home/leos/.local/lib/python3.6/site-packages/ray/node.py:649: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2020-07-09_18-39-33_655807_3419/logs/redis-shard_0.err' mode='a' encoding='UTF-8'>

/home/leos/.local/lib/python3.6/site-packages/ray/node.py:649: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2020-07-09_18-39-33_655807_3419/logs/redis-shard_0.out' mode='a' encoding='UTF-8'>

/home/leos/.local/lib/python3.6/site-packages/ray/node.py:649: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2020-07-09_18-39-33_655807_3419/logs/redis.err' mode='a' encoding='UTF-8'>

/home/leos/.local/lib/python3.6/site-packages/ray/node.py:649: ResourceWarning:

unclosed file <_io.TextIOWrapper

{'node_ip_address': '192.168.10.221',
 'raylet_ip_address': '192.168.10.221',
 'redis_address': '192.168.10.221:55482',
 'object_store_address': '/tmp/ray/session_2020-07-09_18-39-33_655807_3419/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-09_18-39-33_655807_3419/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-09_18-39-33_655807_3419'}

In [6]:
inf = Inferer.remote(
    batch_size = 6,
    patch_size = 224,
    model = model,
    n_classes = 2,
    smoothen=False,
    dataset='consep'
)

In [7]:
infobj = inf.run.remote()

2020-07-09 18:39:43,440	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::Inferer.__init__() (pid=3542, ip=192.168.10.221)
  File "python/ray/_raylet.pyx", line 427, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 444, in ray._raylet.execute_task
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 317, in deserialize_objects
    self._deserialize_object(data, metadata, object_id))
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 257, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata)
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 238, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 226, in _deserialize_pickle5_data
    obj = pickle.loads(in_band)
ModuleNotFoundError: No module named 'dl'
2020-07-09 

In [7]:
inf.plot_histograms()

TypeError: Actor methods cannot be called directly. Instead of running 'object.plot_histograms()', try 'object.plot_histograms.remote()'.

In [ ]:
inf.save_outputs('pöö/pö/pöö/image.png')

In [ ]:
inf.plot_predictions()

2020-07-09 18:29:44,911	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::Inferer.__init__() (pid=3072, ip=192.168.10.221)
  File "python/ray/_raylet.pyx", line 427, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 444, in ray._raylet.execute_task
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 317, in deserialize_objects
    self._deserialize_object(data, metadata, object_id))
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 257, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata)
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 238, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/home/leos/.local/lib/python3.6/site-packages/ray/serialization.py", line 226, in _deserialize_pickle5_data
    obj = pickle.loads(in_band)
ModuleNotFoundError: No module named 'dl'
2020-07-09 